<a href="https://colab.research.google.com/github/Nithinps021/Project_IISU/blob/main/Bert_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [2]:
import json
import numpy as np
import random
import tensorflow as tf



In [4]:
data_file = open('is_train.json').read()
data_set = json.loads(data_file)
random.shuffle(data_set)

In [6]:
seq_len=100
num_samples=len(data_set)

xids = np.zeros((num_samples,seq_len))
xmask = np.zeros((num_samples,seq_len))

xids.shape

(15000, 100)

In [7]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

for i,data in enumerate(data_set):
  tokens=tokenizer.encode_plus(data[0],max_length=seq_len,truncation=True,padding='max_length',add_special_tokens=True,return_tensors='tf')
  xids[i,:]=tokens['input_ids']
  xmask[i,:]=tokens['attention_mask']
  print(tokens)

Streaming output truncated to the last 5000 lines.
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 100), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>}
{'input_ids': <tf.Tensor: shape=(1, 100), dtype=int32, numpy=
array([[ 101, 1132, 1128, 1126, 4315, 1825,  117, 1137,  170, 1182,  102,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,  

In [8]:
classes=[]
for i,j in data_set:
  if j not in classes:
    classes.append(j)
labels = np.zeros((num_samples,len(classes)))
for i,data in enumerate(data_set):
  labels[i,classes.index(data[1])]=1


In [9]:
dataset = tf.data.Dataset.from_tensor_slices((xids,xmask,labels))

def map_function(xids,xmask,labels):
  return {'input_ids':xids,'attention_mask':xmask},labels

dataset=dataset.map(map_function)


In [10]:
batch_size=100
dataset =dataset.shuffle(10000).batch(batch_size,drop_remainder=True)

In [11]:
split=0.9
size = int((num_samples/batch_size)*split)
train=dataset.take(size)
val_data=dataset.skip(size)

In [12]:
from transformers import TFBertModel

bert = TFBertModel.from_pretrained('bert-base-cased')

bert.summary()

Downloading:   0%|          | 0.00/502M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "tf_bert_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  108310272 
                                                                 
Total params: 108,310,272
Trainable params: 108,310,272
Non-trainable params: 0
_________________________________________________________________


In [13]:
input_ids= tf.keras.layers.Input(shape=(seq_len,),name='input_ids',dtype='int32')
mask=tf.keras.layers.Input(shape=(seq_len,),name='attention_mask',dtype='int32')

embedding= bert.bert(input_ids,attention_mask=mask)[1]
x=tf.keras.layers.Dense(1024,activation='relu')(embedding)
y=tf.keras.layers.Dense(len(classes),activation='softmax',name='output')(x)

model=tf.keras.Model(inputs=[input_ids,mask],outputs=y)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 100)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 100)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 100,                                           

In [14]:
optimizer = tf.keras.optimizers.Adam(lr=1e-5, decay=1e-6)
loss = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[acc])


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
history = model.fit(
    train,
    epochs=30,
    validation_data=val_data
)

Epoch 1/30
